# 라쏘(Lasso) 회귀
- 규제 선형 모델: MSE를 최소화하고 회귀계수의 크기를 제어함으로써, 다항 회귀에서의 과적합(overfitting)을 방지
- 라쏘 회귀는 L1 규제를 적용
    - L1 규제: alpha 값을 조정하여 가중치의 값을 규제하며, 이는 L2 규제와 달리 가중치를 0으로 만들 수 있다

In [5]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_boston
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score

boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['PRICE'] = boston.target

y_target = df['PRICE']
X_data = df.drop(['PRICE'], axis=1, inplace=False)

def get_linear_reg_eval(model_name, params=None, X_data_n=None, index_n=None, y_target_n=None):
    coeff_df = pd.DataFrame()
    for param in params:
        if model_name == 'Ridge': model = Ridge(alpha=param)
        elif model_name == 'Lasso': model = Lasso(alpha=param)
        elif model_name == 'ElasticNet': model = ElasticNet(alpha=param, l1_ratio=0.7)
        neg_mse_scores = cross_val_score(model, X_data_n, y_target_n, scoring='neg_mean_squared_error', cv=5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        print('alpha {} : {}'.format(param, avg_rmse))
        model.fit(X_data_n, y_target_n)
        coeff = pd.Series(data=model.coef_, index=index_n)
        colname = 'alpha:' + str(param)
        coeff_df[colname] = coeff
    return coeff_df

lasso_alphas = [0.07, 0.1, 0.5, 1, 3]
coeff_lasso_df = get_linear_reg_eval('Lasso', params=lasso_alphas, X_data_n=X_data, index_n=X_data.columns, y_target_n=y_target)

alpha 0.07 : 5.6122842675266735
alpha 0.1 : 5.615116035266935
alpha 0.5 : 5.669123409594897
alpha 1 : 5.776020813823376
alpha 3 : 6.188763210800905


In [7]:
sort_column = 'alpha:' + str(lasso_alphas[0])
coeff_lasso_df.sort_values(by=sort_column, ascending=False)

,alpha:0.07,alpha:0.1,alpha:0.5,alpha:1,alpha:3
RM,3.789725,3.703202,2.498212,0.949811,0.000000
CHAS,1.434343,0.955190,0.000000,0.000000,0.000000
RAD,0.270936,0.274707,0.277451,0.264206,0.061864
ZN,0.049059,0.049211,0.049544,0.049165,0.037231
B,0.010248,0.010249,0.009469,0.008247,0.006510
NOX,-0.000000,-0.000000,-0.000000,-0.000000,0.000000
AGE,-0.011706,-0.010037,0.003604,0.020910,0.042495
TAX,-0.014290,-0.014570,-0.015442,-0.015212,-0.008602
INDUS,-0.042120,-0.036619,-0.005253,-0.000000,-0.000000
CRIM,-0.098193,-0.097894,-0.083289,-0.063437,-0.000000
